In [1]:
import numpy as np
import sys

In [2]:
lam = 2
sigma2 = 0.1
d = 5
iterations = 50

In [3]:
users = 300
objects = 500

def make_data():
    
    data = np.empty([0,3])
    
    for i in range(users):
        obs = np.random.choice(objects,size=np.random.randint(1, 100), replace=False)
        for j in obs:
            data = np.append(data, np.array([[i, j, np.random.randint(1, 6)]]), axis=0)
            
    return data

In [4]:
#train_data = np.genfromtxt(sys.argv[1], delimiter = ",")
train_data = make_data()

N1 = int(np.max(np.unique(train_data[:,0]))+1)
N2 = int(np.max(np.unique(train_data[:,1]))+1)

print N1, N2

300 500


In [23]:
def PMF(train_data):
    
    v = np.empty([0, d])
    
    for j in range(N2):
        v = np.append(v, 
                      np.random.multivariate_normal(np.zeros([d]), 
                                                    lam**(-1)*np.eye(d)).reshape(1,-1), 
                      axis=0)
        
    print 'created vs'

    Us = np.empty([0,N1,d])
    Vs = np.empty([0,N2,d])
    obj_funcs = np.empty([0,1])

    for t in range(iterations):
        
        u = np.empty([N1, d])

        for i in range(N1):
            
            v_sum = np.zeros([d,d])
            mv_sum = np.zeros([d])
            
            for rating_index in np.where(train_data[:,0]==i)[0]:
                j = int(train_data[rating_index, 1])
                v_sum += np.matmul(v[j].reshape(-1,1), v[j].reshape(1,-1)).reshape(d,d)
                    
                row = np.intersect1d(np.where(train_data[:,0]==i), np.where(train_data[:,1]==j))
                mv_sum += (train_data[row, 2]*v[j])
            
            u[i] = np.matmul(np.linalg.matrix_power(lam*sigma2*np.eye(d) + v_sum, -1),mv_sum.reshape(-1,1)).reshape(1,-1)
            
        print 'updated us'
        
        for j in range(N2):
            
            u_sum = np.zeros([d,d])
            mu_sum = np.zeros([d])
            
            for rating_index in np.where(train_data[:,1]==j)[0]:
                i = int(train_data[rating_index, 0])
                u_sum += np.matmul(u[i].reshape(-1,1), u[i].reshape(1,-1)).reshape(d,d)
                    
                row = np.intersect1d(np.where(train_data[:,0]==i), np.where(train_data[:,1]==j))                        
                mu_sum += (train_data[row, 2]*u[i])
            
            v[j] = np.matmul(np.linalg.matrix_power(lam*sigma2*np.eye(d) + u_sum, -1),mu_sum.reshape(-1,1)).reshape(1,-1)
    
        print 'updated vs'
    
        err_sum = 0
        U_sum = 0
        V_sum = 0

        for i in range(N1):
            U_sum += lam/2*np.linalg.norm(u[i])**2
            
        for j in range(N2):
            V_sum += lam/2*np.linalg.norm(v[i])**2

        for rating in train_data:         
            err_sum += 1/(2*sigma2)*(rating[2]-np.matmul((u[int(rating[0])]).reshape(1,-1),
                                                         (v[int(rating[1])]).reshape(-1,1)))**2
        
        obj_func = -err_sum-U_sum-V_sum
        
        print 'done objective function'
        
        Us = np.append(Us, u.reshape(1,N1,d), axis=0)
        Vs = np.append(Vs, v.reshape(1,N2,d), axis=0)
        obj_funcs = np.append(obj_funcs, obj_func.reshape(1,1), axis=0)
        print 'iteration', t, 'done'
        
    return (obj_funcs, Us, Vs)

In [22]:
L, U_matrices, V_matrices = PMF(train_data)

np.savetxt("objective.csv", L, delimiter=",")

np.savetxt("U-10.csv", U_matrices[9], delimiter=",")
np.savetxt("U-25.csv", U_matrices[24], delimiter=",")
np.savetxt("U-50.csv", U_matrices[49], delimiter=",")

np.savetxt("V-10.csv", V_matrices[9], delimiter=",")
np.savetxt("V-25.csv", V_matrices[24], delimiter=",")
np.savetxt("V-50.csv", V_matrices[49], delimiter=",")

created vs
updated us
updated vs
done objective function
iteration 0 done
updated us
updated vs


KeyboardInterrupt: 

In [8]:
a = np.random.randint(5, size=99).reshape(-1,3)

#print a
b= np.where(a[:,0]==2)
c= np.where(a[:,1]==3)

print b, c

print np.intersect1d(b, c)

(array([ 2,  9, 13, 16, 19, 28, 31]),) (array([ 7, 28]),)
[28]


In [ ]:
def PMF(train_data):
    
    v = np.empty([0, d])
    
    for j in range(N2):
        v = np.append(v, 
                      np.random.multivariate_normal(np.zeros([d]), 
                                                    lam**(-1)*np.eye(d)).reshape(1,-1), 
                      axis=0)
        
    print 'created vs'

    Us = np.empty([0,N1,d])
    Vs = np.empty([0,N2,d])
    obj_funcs = np.empty([0,1])

    for t in range(iterations):
        
        u = np.empty([N1, d])

        for i in range(N1):
            
            v_sum = np.empty([0,d,d])
            mv_sum = np.empty([0,d])
            
            
            for rating in train_data:
                if rating[0] == i:
                    j = int(rating[1])
                    v_sum = np.append(v_sum, np.matmul(v[j].reshape(-1,1), v[j].reshape(1,-1)).reshape(1,d,d), axis=0)                        
                    row = np.intersect1d(np.where(train_data[:,0]==i), np.where(train_data[:,1]==j))
                    mv_sum = np.append(mv_sum, (train_data[row, 2]*v[j]).reshape(1,-1), axis=0)

                        
            vvt = np.sum(v_sum, axis=0)
            mv = np.sum(mv_sum, axis=0).reshape(-1,1)
            
            u[i] = np.matmul(np.linalg.matrix_power(lam*sigma2*np.eye(d) + vvt, -1),mv).reshape(1,-1)
            
        print 'updated us'
        
        for j in range(N2):
            
            u_sum = np.empty([0,d,d])
            mu_sum = np.empty([0,d])
            
            for rating in train_data:
                if rating[1] == j:
                    i = int(rating[0])
                    u_sum = np.append(u_sum, np.matmul(u[i].reshape(-1,1), u[i].reshape(1,-1)).reshape(1,d,d), axis=0)
                    row = np.intersect1d(np.where(train_data[:,0]==i), np.where(train_data[:,1]==j))                        
                    mu_sum = np.append(mu_sum, (train_data[row, 2]*u[i]).reshape(1,-1), axis=0)
                        
            uut = np.sum(u_sum, axis=0)
            mu = np.sum(mu_sum, axis=0).reshape(-1,1)
            
            v[j] = np.matmul(np.linalg.matrix_power(lam*sigma2*np.eye(d) + uut, -1),mu).reshape(1,-1)
    
        print 'updated vs'
    
        err_sum = 0
        U_sum = 0
        V_sum = 0

        for i in range(N1):
            U_sum += lam/2*np.linalg.norm(u[i])**2
            
        for j in range(N2):
            V_sum += lam/2*np.linalg.norm(u[i])**2

        for rating in train_data:
            i = int(rating[0])
            j = int(rating[1])
            row = np.intersect1d(np.where(train_data[:,0]==i), np.where(train_data[:,1]==j))
            err_sum += 1/(2*sigma2)*(train_data[row,2]-np.matmul(np.transpose(u[i]),v[j]))**2
    
        err = np.sum(err_sum, axis=0)
        
        obj_func = -err-U_sum-V_sum
        
        print 'done objective function'
        
        Us = np.append(Us, u.reshape(1,N1,d), axis=0)
        Vs = np.append(Vs, v.reshape(1,N2,d), axis=0)
        obj_funcs = np.append(obj_funcs, obj_func.reshape(1,1), axis=0)
        print 'iteration', t, 'done'
        
    return (obj_funcs, Us, Vs)

In [5]:
#M = np.zeros([N1, N2])

#for i in train_data:
#    M[int(i[0]),int(i[1])] = i[2]

In [16]:
add = np.empty([0,5])
for i in range(50000):
    add = np.append(add, np.arange(5).reshape(1,-1),axis=0)
    
print np.sum(add, axis=0)

[      0.   50000.  100000.  150000.  200000.]


In [18]:
add = np.zeros([1,5])
for i in range(50000):
    add += np.arange(5).reshape(1,-1)
    
print add

[[      0.   50000.  100000.  150000.  200000.]]
